In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
from statsmodels.stats import proportion
from statsmodels.stats.power import NormalIndPower
import warnings
warnings.filterwarnings('ignore')


In [ ]:
# Настройка визуализаций
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
plt.rcParams['figure.figsize'] = (12, 6)
plt.rcParams['font.size'] = 12

In [ ]:
np.random.seed(42)


#df = pd.read_csv('ab_test_data.csv')



In [ ]:
n_users = 10000  
days = 14  
dates = pd.date_range(start='2024-01-01', periods=days, freq='D')
user_ids = range(1, n_users + 1)
groups = np.random.choice(['A', 'B'], size=n_users, p=[0.5, 0.5])

In [ ]:
conversion_probs = np.where(groups == 'A', 0.117, 0.125)
conversions = np.random.binomial(1, conversion_probs)

In [ ]:
revenue = np.zeros(n_users)
purchased_idx = np.where(conversions == 1)[0]
revenue[purchased_idx] = np.random.lognormal(mean=8.5, sigma=1.2, size=len(purchased_idx))

session_duration = np.random.exponential(30, n_users) + 10

In [ ]:
df = pd.DataFrame({
    'user_id': user_ids,
    'date': np.random.choice(dates, n_users),
    'group': groups,
    'conversion': conversions,
    'revenue': revenue,
    'session_duration': session_duration
})

In [ ]:
traffic_sources = ['organic', 'direct', 'social', 'email', 'paid']
df['traffic_source'] = np.random.choice(traffic_sources, n_users, p=[0.4, 0.3, 0.15, 0.1, 0.05])

In [ ]:
print("Размер датасета:", df.shape)
print("\n Первые 5 строк:")
print(df.head())

print("\nБазовая информация о данных:")
print(df.info())

print("\n Описательная статистика:")
print(df.describe())

In [ ]:
print("Проверка распределения по группам:")
group_distribution = df['group'].value_counts()
print(group_distribution)
print(f"\nДоля группы A: {group_distribution['A']/len(df):.2%}")
print(f"Доля группы B: {group_distribution['B']/len(df):.2%}")

In [ ]:
print("\n Проверка сплитования по источникам трафика:")
traffic_split = pd.crosstab(df['traffic_source'], df['group'], normalize='columns')
print(traffic_split * 100)

In [ ]:
chi2, p_value, _, _ = stats.chi2_contingency(pd.crosstab(df['traffic_source'], df['group']))
print(f"\n Хи-квадрат тест на равномерность распределения:")
print(f"χ² = {chi2:.3f}, p-value = {p_value:.3f}")
print("Распределение равномерное" if p_value > 0.05 else "Возможны проблемы со сплитованием")

# %%
print("\n Проверка временного распределения:")
df['date'] = pd.to_datetime(df['date'])
df['day_of_week'] = df['date'].dt.day_name()
df['is_weekend'] = df['date'].dt.dayofweek >= 5

In [ ]:
weekend_split = pd.crosstab(df['is_weekend'], df['group'], normalize='columns')
print("Распределение по выходным/будням (%):")
print(weekend_split * 100)

In [ ]:
def calculate_ci(data, confidence=0.95):
    mean = np.mean(data)
    sem = stats.sem(data)
    margin = sem * stats.t.ppf((1 + confidence) / 2., len(data)-1)
    return mean, mean - margin, mean + margin

In [ ]:
metrics = {}

for group in ['A', 'B']:
    group_data = df[df['group'] == group]
    n_users = len(group_data)
    n_conversions = group_data['conversion'].sum()
    conversion_rate = n_conversions / n_users
    total_revenue = group_data['revenue'].sum()
    avg_revenue_per_user = total_revenue / n_users
    avg_revenue_per_paying = total_revenue / n_conversions if n_conversions > 0 else 0
    
    avg_session_duration = group_data['session_duration'].mean()
    ci_low, ci_high = proportion.proportion_confint(
        n_conversions, n_users, alpha=0.05, method='wilson'
    )
    
    metrics[group] = {
        'n_users': n_users,
        'n_conversions': n_conversions,
        'conversion_rate': conversion_rate,
        'conversion_ci_low': ci_low,
        'conversion_ci_high': ci_high,
        'total_revenue': total_revenue,
        'avg_revenue_per_user': avg_revenue_per_user,
        'avg_revenue_per_paying': avg_revenue_per_paying,
        'avg_session_duration': avg_session_duration
    }

metrics_df = pd.DataFrame(metrics).T
print("📊 Основные метрики по группам:")
print(metrics_df[['n_users', 'n_conversions', 'conversion_rate', 
                  'avg_revenue_per_user', 'avg_session_duration']])


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))


In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(16, 12))


In [ ]:
ax1 = axes[0, 0]
groups = ['A', 'B']
conversion_rates = [metrics[g]['conversion_rate'] for g in groups]
ci_lows = [metrics[g]['conversion_ci_low'] for g in groups]
ci_highs = [metrics[g]['conversion_ci_high'] for g in groups]

bars = ax1.bar(groups, conversion_rates, color=['#3498db', '#2ecc71'], alpha=0.7)
ax1.errorbar(groups, conversion_rates, 
            yerr=[np.array(conversion_rates) - ci_lows, ci_highs - np.array(conversion_rates)],
            fmt='none', color='black', capsize=10, linewidth=2)


In [ ]:
for i, (bar, rate) in enumerate(zip(bars, conversion_rates)):
    ax1.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001,
            f'{rate:.2%}', ha='center', va='bottom', fontsize=12, fontweight='bold')

ax1.set_ylabel('Конверсия (%)', fontsize=12)
ax1.set_title('Конверсия в покупку по группам (с 95% доверительными интервалами)', 
              fontsize=14, fontweight='bold', pad=20)
ax1.set_ylim(0, max(conversion_rates) * 1.2)
ax1.grid(True, alpha=0.3)

In [ ]:
ax2 = axes[0, 1]
daily_conversion = df.groupby(['date', 'group'])['conversion'].mean().unstack()
daily_conversion.plot(ax=ax2, linewidth=3, marker='o')
ax2.set_ylabel('Конверсия (%)', fontsize=12)
ax2.set_title('Динамика конверсии по дням', fontsize=14, fontweight='bold', pad=20)
ax2.legend(title='Группа')
ax2.grid(True, alpha=0.3)
ax2.set_xticklabels(ax2.get_xticklabels(), rotation=45)

In [ ]:
ax3 = axes[1, 0]
revenue_data_a = df[df['group'] == 'A']['revenue']
revenue_data_b = df[df['group'] == 'B']['revenue']

paying_a = revenue_data_a[revenue_data_a > 0]
paying_b = revenue_data_b[revenue_data_b > 0]

ax3.hist([paying_a, paying_b], bins=30, alpha=0.7, 
         label=['Группа A', 'Группа B'], density=True)
ax3.set_xlabel('Доход с пользователя (руб.)', fontsize=12)
ax3.set_ylabel('Плотность', fontsize=12)
ax3.set_title('Распределение дохода с покупателей', fontsize=14, fontweight='bold', pad=20)
ax3.legend()
ax3.grid(True, alpha=0.3)

In [ ]:
ax4 = axes[1, 1]
metrics_comparison = pd.DataFrame({
    'Конверсия': [metrics['A']['conversion_rate'], metrics['B']['conversion_rate']],
    'Ср. чек': [metrics['A']['avg_revenue_per_paying'], metrics['B']['avg_revenue_per_paying']],
    'Доход на пользователя': [metrics['A']['avg_revenue_per_user'], metrics['B']['avg_revenue_per_user']]
}, index=['Группа A', 'Группа B'])



In [ ]:
metrics_normalized = metrics_comparison / metrics_comparison.max()

In [ ]:
angles = np.linspace(0, 2*np.pi, len(metrics_normalized.columns), endpoint=False).tolist()
angles += angles[:1]  # Замыкаем круг

for idx, group in enumerate(metrics_normalized.index):
    values = metrics_normalized.loc[group].values.tolist()
    values += values[:1]
    ax4.plot(angles, values, 'o-', linewidth=2, label=group)
    ax4.fill(angles, values, alpha=0.1)

ax4.set_xticks(angles[:-1])
ax4.set_xticklabels(metrics_normalized.columns)
ax4.set_title('Сравнение метрик (нормализованные значения)', 
              fontsize=14, fontweight='bold', pad=20)
ax4.legend(loc='upper right')
ax4.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
print("СТАТИСТИЧЕСКИЙ АНАЛИЗ РАЗЛИЧИЙ")
print("="*50)

# 6.1 Z-тест для разницы долей (конверсия)
print("\n1. Z-тест для разницы конверсий:")
success_a, nobs_a = metrics['A']['n_conversions'], metrics['A']['n_users']
success_b, nobs_b = metrics['B']['n_conversions'], metrics['B']['n_users']

In [ ]:
z_score, p_value = proportion.proportions_ztest(
    [success_a, success_b], 
    [nobs_a, nobs_b],
    alternative='smaller'
)

print(f"   Конверсия A: {metrics['A']['conversion_rate']:.3%}")
print(f"   Конверсия B: {metrics['B']['conversion_rate']:.3%}")
print(f"   Абсолютная разница: {(metrics['B']['conversion_rate'] - metrics['A']['conversion_rate']):.3%}")
print(f"   Относительная разница: {(metrics['B']['conversion_rate']/metrics['A']['conversion_rate'] - 1):.2%}")
print(f"   Z-статистика: {z_score:.3f}")
print(f"   P-value: {p_value:.4f}")

In [ ]:
ci_low, ci_high = proportion.confint_proportions_2indep(
    success_b, nobs_b, success_a, nobs_a, method='wald'
)
print(f"   95% ДИ для разницы: [{ci_low:.4f}, {ci_high:.4f}]")

print(f"\n   Вывод: Разница {'НЕ ' if p_value > 0.05 else ''}является статистически значимой (α=0.05)")

In [ ]:
# T-тест для среднего чека (только покупатели)
print("\n2. T-тест для среднего чека:")

paying_users_a = df[(df['group'] == 'A') & (df['conversion'] == 1)]['revenue']
paying_users_b = df[(df['group'] == 'B') & (df['conversion'] == 1)]['revenue']

if len(paying_users_a) > 1 and len(paying_users_b) > 1:
    t_stat, p_value_t = stats.ttest_ind(paying_users_a, paying_users_b, equal_var=False)
    
    print(f"   Средний чек A: {metrics['A']['avg_revenue_per_paying']:.2f} руб.")
    print(f"   Средний чек B: {metrics['B']['avg_revenue_per_paying']:.2f} руб.")
    print(f"   Разница: {(metrics['B']['avg_revenue_per_paying'] - metrics['A']['avg_revenue_per_paying']):.2f} руб.")
    print(f"   T-статистика: {t_stat:.3f}")
    print(f"   P-value: {p_value_t:.4f}")
    print(f"    Вывод: Разница в среднем чеке {'НЕ ' if p_value_t > 0.05 else ''}является статистически значимой")
else:
    print("   Недостаточно данных для t-теста")


In [ ]:
# Расчет мощности теста
print("\n3. Анализ мощности теста:")

effect_size = proportion.proportion_effectsize(
    metrics['A']['conversion_rate'], 
    metrics['B']['conversion_rate']
)

power_analysis = NormalIndPower()
required_n = power_analysis.solve_power(
    effect_size=effect_size,
    power=0.8,
    alpha=0.05,
    ratio=metrics['B']['n_users']/metrics['A']['n_users']
)

actual_power = power_analysis.solve_power(
    effect_size=effect_size,
    nobs1=metrics['A']['n_users'],
    alpha=0.05,
    ratio=metrics['B']['n_users']/metrics['A']['n_users']
)

print(f"   Размер эффекта (Cohen's h): {effect_size:.3f}")
print(f"   Требуемый размер выборки для мощности 80%: {int(required_n)} на группу")
print(f"   Фактическая мощность теста: {actual_power:.2%}")
print(f"   Фактический размер выборки: {metrics['A']['n_users']} (A), {metrics['B']['n_users']} (B)")

if actual_power < 0.8:
    print("     Внимание: Мощность теста ниже рекомендуемого уровня 80%")
    print("   Рекомендация: Увеличить длительность теста или размер выборки")


In [ ]:
# ## Анализ чувствительности и дополнительные проверки

In [ ]:
# Анализ по сегментам
print("📈 АНАЛИЗ ПО СЕГМЕНТАМ")
print("="*50)

segments = ['traffic_source', 'is_weekend']
for segment in segments:
    print(f"\n По сегменту: {segment}")
    segment_results = []
    
    for value in df[segment].unique():
        segment_data = df[df[segment] == value]
        if len(segment_data) < 50:  # Пропускаем маленькие сегменты
            continue
            
        conv_a = segment_data[segment_data['group'] == 'A']['conversion'].mean()
        conv_b = segment_data[segment_data['group'] == 'B']['conversion'].mean()
        
        if not (np.isnan(conv_a) or np.isnan(conv_b)):
            segment_results.append({
                'segment_value': value,
                'conv_a': conv_a,
                'conv_b': conv_b,
                'abs_diff': conv_b - conv_a,
                'rel_diff': (conv_b / conv_a - 1) if conv_a > 0 else 0
            })
    
    results_df = pd.DataFrame(segment_results)
    if not results_df.empty:
        print(results_df.sort_values('abs_diff', ascending=False).to_string(index=False))
        
        # Проверяем, есть ли сегменты с противоположным эффектом
        mixed_effects = (results_df['abs_diff'] > 0).any() and (results_df['abs_diff'] < 0).any()
        if mixed_effects:
            print("     Обнаружены сегменты с противоположным эффектом!")


In [ ]:
# Анализ воронки (если есть данные о действиях пользователей)
print("\n📊 АНАЛИЗ ПОВЕДЕНЧЕСКИХ МЕТРИК")
print("="*50)

print("1. Время на сайте:")
duration_a = df[df['group'] == 'A']['session_duration']
duration_b = df[df['group'] == 'B']['session_duration']

t_stat_dur, p_val_dur = stats.ttest_ind(duration_a, duration_b, equal_var=False)
print(f"   Группа A: {duration_a.mean():.1f} мин.")
print(f"   Группа B: {duration_b.mean():.1f} мин.")
print(f"   P-value: {p_val_dur:.4f}")
print(f"   Вывод: {'Есть' if p_val_dur < 0.05 else 'Нет'} значимой разницы во времени на сайте")

In [ ]:
 ## 8. Бизнес-оценка результатов

In [ ]:

print(" БИЗНЕС-ОЦЕНКА РЕЗУЛЬТАТОВ")
print("="*50)

# Расчет потенциального эффекта
baseline_conversion = metrics['A']['conversion_rate']
new_conversion = metrics['B']['conversion_rate']
monthly_users = 100000  # примерное количество пользователей в месяц
avg_order_value = metrics['A']['avg_revenue_per_paying']

if p_value < 0.05:  # Если эффект значим
    additional_conversions = monthly_users * 0.5 * (new_conversion - baseline_conversion)
    additional_revenue = additional_conversions * avg_order_value
    
    print(f"\n При внедрении изменений:")
    print(f"   Дополнительных покупок в месяц: {additional_conversions:.0f}")
    print(f"   Дополнительный доход в месяц: {additional_revenue:,.0f} руб.")
    print(f"   Дополнительный доход в год: {additional_revenue * 12:,.0f} руб.")
    
    # Расчет ROI (если известна стоимость внедрения)
    implementation_cost = 500000  # примерная стоимость
    if implementation_cost > 0:
        months_to_roi = implementation_cost / additional_revenue
        print(f"   Срок окупаемости: {months_to_roi:.1f} месяцев")
else:
    print(f"\n  Статистически значимого эффекта не обнаружено")
    print(f"   Внедрение изменений может не привести к росту выручки")
    
    # Оценка риска потерь
    worst_case_scenario = ci_low  # нижняя граница ДИ для разницы
    if worst_case_scenario < 0:
        potential_loss = monthly_users * 0.5 * abs(worst_case_scenario) * avg_order_value
        print(f"   Потенциальные потери (worst-case): {potential_loss:,.0f} руб. в месяц")


In [ ]:
 ## 9. Выводы и рекомендации


print(" ИТОГОВЫЕ ВЫВОДЫ И РЕКОМЕНДАЦИИ")
print("="*50)

print(f"\n Статистические выводы:")
print(f"   1. Конверсия в группе B выше на {(new_conversion/baseline_conversion - 1)*100:.1f}%")
print(f"   2. P-value = {p_value:.4f} → {'Статистически значимо' if p_value < 0.05 else 'Не статистически значимо'}")
print(f"   3. Мощность теста: {actual_power:.1%} ({'достаточно' if actual_power >= 0.8 else 'недостаточно'})")

print(f"\n📈 Бизнес-выводы:")
if p_value < 0.05:
    print("    РЕКОМЕНДАЦИЯ: Внедрить новую рекомендательную систему")
    print(f"   Ожидаемый эффект: +{additional_revenue:,.0f} руб./мес.")
else:
    print("     РЕКОМЕНДАЦИЯ: Отклонить изменения или провести новый тест")
    print(f"   Причина: Недостаточно доказательств эффективности")

print(f"\n🔍 Рекомендации по следующим шагам:")
if p_value < 0.05:
    print("   1. Запланировать постепенный rollout (10% → 50% → 100%)")
    print("   2. Мониторить ключевые метрики после внедрения")
    print("   3. Провести A/A тест через месяц для проверки стабильности")
else:
    print("   1. Проанализировать причины возможной неэффективности")
    print("   2. Провести качественное исследование (юзабилити-тесты)")
    print("   3. Разработать новые гипотезы для улучшения")
    
if actual_power < 0.8:
    print("   4. Повторить тест с большей выборкой или длительностью")



# Сохранение результатов в файлы
df.to_csv('ab_test_data.csv', index=False, encoding='utf-8-sig')


In [ ]:
# Сохранение сводной таблицы с метриками
summary_report = pd.DataFrame({
    'Метрика': [
        'Размер группы', 
        'Количество покупок', 
        'Конверсия', 
        'Конверсия (нижняя граница 95% ДИ)',
        'Конверсия (верхняя граница 95% ДИ)',
        'Общий доход',
        'Доход на пользователя',
        'Средний чек'
    ],
    'Группа A': [
        metrics['A']['n_users'],
        metrics['A']['n_conversions'],
        f"{metrics['A']['conversion_rate']:.3%}",
        f"{metrics['A']['conversion_ci_low']:.3%}",
        f"{metrics['A']['conversion_ci_high']:.3%}",
        f"{metrics['A']['total_revenue']:,.0f} руб.",
        f"{metrics['A']['avg_revenue_per_user']:.2f} руб.",
        f"{metrics['A']['avg_revenue_per_paying']:.2f} руб."
    ],
    'Группа B': [
        metrics['B']['n_users'],
        metrics['B']['n_conversions'],
        f"{metrics['B']['conversion_rate']:.3%}",
        f"{metrics['B']['conversion_ci_low']:.3%}",
        f"{metrics['B']['conversion_ci_high']:.3%}",
        f"{metrics['B']['total_revenue']:,.0f} руб.",
        f"{metrics['B']['avg_revenue_per_user']:.2f} руб.",
        f"{metrics['B']['avg_revenue_per_paying']:.2f} руб."
    ],
    'Разница (B - A)': [
        f"{metrics['B']['n_users'] - metrics['A']['n_users']}",
        f"{metrics['B']['n_conversions'] - metrics['A']['n_conversions']}",
        f"{(metrics['B']['conversion_rate'] - metrics['A']['conversion_rate']):.3%}",
        '-',
        '-',
        f"{(metrics['B']['total_revenue'] - metrics['A']['total_revenue']):,.0f} руб.",
        f"{(metrics['B']['avg_revenue_per_user'] - metrics['A']['avg_revenue_per_user']):.2f} руб.",
        f"{(metrics['B']['avg_revenue_per_paying'] - metrics['A']['avg_revenue_per_paying']):.2f} руб."
    ]
})

print("\n Сводный отчет по тесту:")
print(summary_report.to_string(index=False))

# 2. Сохранение в CSV
summary_report.to_csv('ab_test_summary.csv', index=False, encoding='utf-8-sig')

# 3. Сохранение графиков
fig, ax = plt.subplots(figsize=(10, 6))
x = np.arange(len(summary_report))
width = 0.35

# График для визуализации ключевых метрик
key_metrics = ['Конверсия', 'Доход на пользователя', 'Средний чек']
key_data = summary_report[summary_report['Метрика'].isin(key_metrics)]

for i, metric in enumerate(key_metrics):
    metric_data = key_data[key_data['Метрика'] == metric]
    if not metric_data.empty:
        # Извлекаем числовые значения (убираем текст)
        val_a = float(metric_data['Группа A'].iloc[0].replace(' руб.', '').replace('%', '').replace(' руб', ''))
        val_b = float(metric_data['Группа B'].iloc[0].replace(' руб.', '').replace('%', '').replace(' руб', ''))
        
        ax.bar(i - width/2, val_a, width, label='Группа A' if i == 0 else "", 
               color='#3498db', alpha=0.7)
        ax.bar(i + width/2, val_b, width, label='Группа B' if i == 0 else "", 
               color='#2ecc71', alpha=0.7)

ax.set_xlabel('Метрики', fontsize=12)
ax.set_ylabel('Значение', fontsize=12)
ax.set_title('Сравнение ключевых метрик A/B теста', fontsize=14, fontweight='bold')
ax.set_xticks(range(len(key_metrics)))
ax.set_xticklabels(key_metrics, rotation=45, ha='right')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('ab_test_comparison.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n Результаты сохранены:")
print("   - ab_test_summary.csv - сводный отчет")
print("   - ab_test_comparison.png - график сравнения")
print("   - Все визуализации доступны выше в ноутбуке")